In [1]:
import pandas as pd

In [2]:
ratingpath='/home/swatilathwal/Documents/Untitled Folder/spark-training-master/data/movielens/medium/ratings.dat'

In [3]:
file= pd.read_table(ratingpath,sep='::',names=['UserID','MovieID','Rating','Timestamp'],encoding='iso-8859-15')
#UserID::MovieID::Rating::Timestamp

<ipython-input-3-e040785c172c>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  file= pd.read_table(ratingpath,sep='::',names=['UserID','MovieID','Rating','Timestamp'],encoding='iso-8859-15')


In [4]:
file

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [5]:
file.drop('Timestamp', axis=1, inplace=True)

In [6]:
from pyspark.ml.recommendation import ALS 

In [7]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [8]:
from pyspark.ml.evaluation import RegressionEvaluator

In [9]:
from pyspark import SparkContext

In [10]:
from pyspark import SQLContext

In [11]:
sc=SparkContext()
sqlctx= SQLContext(sc)


In [12]:
SpData= sqlctx.createDataFrame(file)

In [13]:
SpData

DataFrame[UserID: bigint, MovieID: bigint, Rating: bigint]

In [14]:
(train, test)= SpData.randomSplit([0.70,0.30], seed=123)

In [15]:
test.show()

+------+-------+------+
|UserID|MovieID|Rating|
+------+-------+------+
|     1|    150|     5|
|     1|    588|     4|
|     1|    608|     4|
|     1|    745|     3|
|     1|    783|     4|
|     1|   1028|     5|
|     1|   1197|     3|
|     1|   1566|     4|
|     1|   1721|     4|
|     1|   1961|     5|
|     1|   1962|     4|
|     1|   2018|     4|
|     1|   2321|     3|
|     1|   2355|     5|
|     1|   2797|     4|
|     1|   2804|     5|
|     2|    110|     5|
|     2|    235|     3|
|     2|    292|     3|
|     2|    318|     5|
+------+-------+------+
only showing top 20 rows



Creating ALS Model
find best ranks, lambdas, numIters for best rmse

In [16]:
als= ALS(userCol='UserID', itemCol='MovieID', ratingCol='Rating', nonnegative=True, implicitPrefs= False)

In [17]:
type(als)

pyspark.ml.recommendation.ALS

Pararmeter Tuning

In [18]:
param_grid= ParamGridBuilder().addGrid(als.rank,[5,10]).addGrid(als.maxIter,[10,15,20]).addGrid(als.regParam,[0.01,0.03]).build()

In [19]:
evaluation= RegressionEvaluator(metricName='rmse', labelCol='Rating', predictionCol='prediction')

In [20]:
len(param_grid)

12

In [21]:
cvmodel= CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluation, numFolds=5)

In [22]:
print(cvmodel)

CrossValidator_5febd0a77c1c


In [23]:
model=cvmodel.fit(train)

In [24]:
final_model= model.bestModel

In [25]:
type(final_model)

pyspark.ml.recommendation.ALSModel

In [26]:
final_model

ALSModel: uid=ALS_436c8b692512, rank=5

In [27]:
model.bestModel.rank

5

In [28]:
final_model._java_obj.parent().getMaxIter()

10

In [29]:
final_model._java_obj.parent().getRegParam()

0.01

In [30]:
pred=final_model.transform(test)

In [32]:
pred.na.drop()

DataFrame[UserID: bigint, MovieID: bigint, Rating: bigint, prediction: float]

In [33]:
pred=pred.filter(pred.prediction!=float('nan'))

In [34]:
pred.show()

+------+-------+------+----------+
|UserID|MovieID|Rating|prediction|
+------+-------+------+----------+
|   673|    148|     5| 2.8433936|
|  4169|    148|     3| 2.3893642|
|  4784|    148|     3|  3.078099|
|  1242|    148|     3|  2.154727|
|  1069|    148|     2| 2.6888309|
|  1605|    148|     2| 2.2311416|
|   216|    148|     2|  3.007984|
|   752|    148|     4|  3.428245|
|  2507|    148|     4| 2.5319428|
|  3053|    148|     3| 2.3497472|
|   970|    463|     3|  2.529545|
|  4858|    463|     3| 2.5818613|
|    26|    463|     3| 3.0735722|
|  2629|    463|     4| 3.3562837|
|  4040|    463|     1| 1.8807111|
|  3753|    463|     2|  2.984128|
|  5306|    463|     2| 2.5091171|
|  1069|    463|     2| 1.9010103|
|  3717|    463|     2| 3.1081138|
|   524|    463|     3| 2.3744874|
+------+-------+------+----------+
only showing top 20 rows



In [35]:
rmse= evaluation.evaluate(pred)

In [36]:
print(rmse)

0.8748213132371326
